In [44]:
import pandas as pd
import json
import csv
import numpy as np
import psycopg2
import ast
from sqlalchemy import create_engine, text

In [45]:
pd.set_option('display.max_columns', None)
compounds_df = pd.read_csv('homePage_data.csv')
compounds_df.head()

,id,compound_name,is_super,slug,properties_count,image_url,min_price,max_price,min_unit_area,max_unit_area,available_bathrooms,available_bedrooms,developer_id,developer_name,developer_logo,area_id,area_name,sponsored,max_installment_years,min_down_payment,min_ready_by,properties_ids,sum_10_properties_min_price,lat,long,has_offer,has_launches,is_launch,offer_title,limited_time_offer,property_types,property_types_count
0,195,Mountain View ICity New Cairo,True,195-mountain-view-icity-new-cairo,7,https://s3.eu-central-1.amazonaws.com/prod.ima...,7584822,14559080,130,295,[3],[3],6,DMG,https://s3.eu-central-1.amazonaws.com/prod.ima...,2,New Cairo,0,9.0,758483.0,2027-01-01,"[35486, 35487, 35489, 35490, 35495, 35497, 35502]",117896389.0,30.065182,31.593508,False,False,False,"5% Down Payment , instalments up to 9 years",True,"['Apartment', 'Villa', 'Townhouse', 'Duplex', ...","[{'id': 21, 'name': 'Apartment', 'count': 41, ..."
1,274,ZED,True,274-zed-el-sheikh-zayed-ora,7,https://s3.eu-central-1.amazonaws.com/prod.ima...,10273202,20284992,160,301,"[3, 4]","[3, 4]",85,Ora Developers,https://s3.eu-central-1.amazonaws.com/prod.ima...,26,El Sheikh Zayed,0,8.0,513661.0,2024-10-31,"[41998, 41999, 42002, 42110, 42111, 42217, 42528]",129709414.0,30.046608,31.003101,False,False,False,NaN,True,"['Apartment', 'Duplex', 'Penthouse', 'Studio']","[{'id': 21, 'name': 'Apartment', 'count': 10, ..."
2,202,Bloomfields,True,202-bloomfields-tatweer-misr-mostakbal-city,76,https://s3.eu-central-1.amazonaws.com/prod.ima...,3664000,16210000,70,287,"[1, 2, 3, 4]","[1, 2, 3, 4]",33,Tatweer Misr,https://s3.eu-central-1.amazonaws.com/prod.ima...,10,Mostakbal City,0,8.0,209350.0,2023-12-30,"[17436, 31836, 31839, 31844, 31847, 31848, 318...",35147621.0,30.070208,31.649590,False,False,False,NaN,True,"['Apartment', 'Duplex', 'Studio']","[{'id': 21, 'name': 'Apartment', 'count': 76, ..."
3,223,Badya,True,223-badya-palm-hills,42,https://s3.eu-central-1.amazonaws.com/prod.ima...,4782600,31084000,107,499,"[2, 3, 4, 6, 7, 8]","[2, 3, 4, 5, 6]",16,Palm Hills Developments,https://s3.eu-central-1.amazonaws.com/prod.ima...,1,6th of October City,0,10.0,0.0,2023-11-29,"[9742, 11180, 13044, 16026, 19171, 19199, 1961...",44446502.0,29.854789,30.907116,False,False,False,NaN,True,"['Apartment', 'Villa', 'Townhouse', 'Penthouse...","[{'id': 21, 'name': 'Apartment', 'count': 48, ..."
4,148,Palm Hills New Cairo,True,148-palm-hills-new-cairo,7,https://s3.eu-central-1.amazonaws.com/prod.ima...,18886000,72968000,190,611,"[4, 7, 8]","[3, 4, 6]",16,Palm Hills Developments,https://s3.eu-central-1.amazonaws.com/prod.ima...,2,New Cairo,0,9.0,944300.0,2022-12-30,"[22676, 23315, 28769, 28770, 32932, 37716, 42215]",487964000.0,30.059658,31.598640,False,False,False,NaN,True,"['Apartment', 'Villa', 'Twinhouse', 'Townhouse']","[{'id': 21, 'name': 'Apartment', 'count': 13, ..."


In [46]:
# insert data in table compound
# Frist handle data to be like we need to insert
developer_table = compounds_df[['developer_id', 'developer_name', 'developer_logo']]
developer_table.head()

,developer_id,developer_name,developer_logo
0,6,DMG,https://s3.eu-central-1.amazonaws.com/prod.ima...
1,85,Ora Developers,https://s3.eu-central-1.amazonaws.com/prod.ima...
2,33,Tatweer Misr,https://s3.eu-central-1.amazonaws.com/prod.ima...
3,16,Palm Hills Developments,https://s3.eu-central-1.amazonaws.com/prod.ima...
4,16,Palm Hills Developments,https://s3.eu-central-1.amazonaws.com/prod.ima...


In [47]:
# connnect to db 
# Replace these with your database details
dbname = "nawy"
user = "nady"
password = "admin"
host = "172.20.0.3"
port = "5432"


# Establish a connection to the database
try:
    connection = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )

except psycopg2.Error as e:
    print("Error connecting to the database:", e)
    exit()


# Create SQLalchemy engine
engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{dbname}")


In [48]:
#check df has the same db tables column name
#cursor = connection.cursor()

# Query to retrieve column names
with engine.connect() as connection:
    query = text("SELECT column_name FROM information_schema.columns WHERE table_name = 'developer' ORDER BY column_name ASC;")
    results = connection.execute(query)

    # Fetch the results
    column_names = results.fetchall()

In [49]:
# check matching betwween df columns vs db columns
columns = sorted(developer_table.columns)
for row, column_name in enumerate(column_names):
    results = column_name[0] == columns[row]
    if results == True:
        pass
    else:
        print(f"{column_name[0]} vs {columns[row]}")

In [50]:
## match column datatypes in db table with column datatypes in df table
column_types = developer_table.dtypes.sort_index()
column_types
with engine.connect() as connection:
    query = text(
        f"select column_name, data_type from information_schema.columns where table_name='developer' ORDER BY column_name"
    )
    # Excute the query
    results = connection.execute(query)

    #fetch the result
    column_info = results.fetchall()
for row, column_ in enumerate(column_info):
    results = column_[0] == columns[row]
    if results == True:
        print(f"{column_[0]:<30} --> {column_[1]:<20}, :: {column_types[row]}")
    else:
        print(f"{column_name[0]} vs {columns[row]}")

developer_id                   --> integer             , :: int64
developer_logo                 --> character varying   , :: object
developer_name                 --> character varying   , :: object


/tmp/ipykernel_46149/3841289215.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"{column_[0]:<30} --> {column_[1]:<20}, :: {column_types[row]}")


In [51]:
developer_table = developer_table.drop_duplicates(subset=['developer_id'], keep='first')
print(len(developer_table))

186


In [52]:
# insert data to database
developer_table.to_sql('developer', engine, if_exists='append', index=False)

186

### laod data to area table
compounds_df.head()

In [53]:
compounds_df.head()

,id,compound_name,is_super,slug,properties_count,image_url,min_price,max_price,min_unit_area,max_unit_area,available_bathrooms,available_bedrooms,developer_id,developer_name,developer_logo,area_id,area_name,sponsored,max_installment_years,min_down_payment,min_ready_by,properties_ids,sum_10_properties_min_price,lat,long,has_offer,has_launches,is_launch,offer_title,limited_time_offer,property_types,property_types_count
0,195,Mountain View ICity New Cairo,True,195-mountain-view-icity-new-cairo,7,https://s3.eu-central-1.amazonaws.com/prod.ima...,7584822,14559080,130,295,[3],[3],6,DMG,https://s3.eu-central-1.amazonaws.com/prod.ima...,2,New Cairo,0,9.0,758483.0,2027-01-01,"[35486, 35487, 35489, 35490, 35495, 35497, 35502]",117896389.0,30.065182,31.593508,False,False,False,"5% Down Payment , instalments up to 9 years",True,"['Apartment', 'Villa', 'Townhouse', 'Duplex', ...","[{'id': 21, 'name': 'Apartment', 'count': 41, ..."
1,274,ZED,True,274-zed-el-sheikh-zayed-ora,7,https://s3.eu-central-1.amazonaws.com/prod.ima...,10273202,20284992,160,301,"[3, 4]","[3, 4]",85,Ora Developers,https://s3.eu-central-1.amazonaws.com/prod.ima...,26,El Sheikh Zayed,0,8.0,513661.0,2024-10-31,"[41998, 41999, 42002, 42110, 42111, 42217, 42528]",129709414.0,30.046608,31.003101,False,False,False,NaN,True,"['Apartment', 'Duplex', 'Penthouse', 'Studio']","[{'id': 21, 'name': 'Apartment', 'count': 10, ..."
2,202,Bloomfields,True,202-bloomfields-tatweer-misr-mostakbal-city,76,https://s3.eu-central-1.amazonaws.com/prod.ima...,3664000,16210000,70,287,"[1, 2, 3, 4]","[1, 2, 3, 4]",33,Tatweer Misr,https://s3.eu-central-1.amazonaws.com/prod.ima...,10,Mostakbal City,0,8.0,209350.0,2023-12-30,"[17436, 31836, 31839, 31844, 31847, 31848, 318...",35147621.0,30.070208,31.649590,False,False,False,NaN,True,"['Apartment', 'Duplex', 'Studio']","[{'id': 21, 'name': 'Apartment', 'count': 76, ..."
3,223,Badya,True,223-badya-palm-hills,42,https://s3.eu-central-1.amazonaws.com/prod.ima...,4782600,31084000,107,499,"[2, 3, 4, 6, 7, 8]","[2, 3, 4, 5, 6]",16,Palm Hills Developments,https://s3.eu-central-1.amazonaws.com/prod.ima...,1,6th of October City,0,10.0,0.0,2023-11-29,"[9742, 11180, 13044, 16026, 19171, 19199, 1961...",44446502.0,29.854789,30.907116,False,False,False,NaN,True,"['Apartment', 'Villa', 'Townhouse', 'Penthouse...","[{'id': 21, 'name': 'Apartment', 'count': 48, ..."
4,148,Palm Hills New Cairo,True,148-palm-hills-new-cairo,7,https://s3.eu-central-1.amazonaws.com/prod.ima...,18886000,72968000,190,611,"[4, 7, 8]","[3, 4, 6]",16,Palm Hills Developments,https://s3.eu-central-1.amazonaws.com/prod.ima...,2,New Cairo,0,9.0,944300.0,2022-12-30,"[22676, 23315, 28769, 28770, 32932, 37716, 42215]",487964000.0,30.059658,31.598640,False,False,False,NaN,True,"['Apartment', 'Villa', 'Twinhouse', 'Townhouse']","[{'id': 21, 'name': 'Apartment', 'count': 13, ..."


In [83]:
area_table = compounds_df[['area_id', 'area_name', 'lat', 'long']]
area_table.head()

,area_id,area_name,lat,long
0,2,New Cairo,30.065182,31.593508
1,26,El Sheikh Zayed,30.046608,31.003101
2,10,Mostakbal City,30.070208,31.649590
3,1,6th of October City,29.854789,30.907116
4,2,New Cairo,30.059658,31.598640


In [84]:
## get area distincit value to insert in area table
print(len(area_table))
area_table = area_table.drop_duplicates(subset='area_id', keep='first')
print(len(area_table))

530
19


In [85]:
#check df has the same db tables column name
#cursor = connection.cursor()

# Query to retrieve column names
with engine.connect() as connection:
    query = text("SELECT column_name FROM information_schema.columns WHERE table_name = 'area' ORDER BY column_name ASC;")
    results = connection.execute(query)

    # Fetch the results
    column_names = results.fetchall()

In [86]:
# check matching between df columns vs db columns
columns = sorted(area_table.columns)
for row, column_name in enumerate(column_names):
    results = column_name[0] == columns[row]
    if results == True:
        pass
    else:
        print(f"{column_name[0]} vs {columns[row]}")

In [88]:
## match column datatypes in db table with column datatypes in df table
column_types = area_table.dtypes.sort_index()
column_types
with engine.connect() as connection:
    query = text(
        f"select column_name, data_type from information_schema.columns where table_name='area' ORDER BY column_name"
    )
    # Excute the query
    results = connection.execute(query)

    #fetch the result
    column_info = results.fetchall()
for row, column_ in enumerate(column_info):
    results = column_[0] == columns[row]
    if results == True:
        print(f"{column_[0]:<30} --> {column_[1]:<20}, :: {column_types[row]}")
    else:
        print(f"{column_name[0]} vs {columns[row]}")

area_id                        --> integer             , :: int64
area_name                      --> character varying   , :: object
lat                            --> double precision    , :: float64
long                           --> double precision    , :: float64


/tmp/ipykernel_46149/2745697743.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"{column_[0]:<30} --> {column_[1]:<20}, :: {column_types[row]}")


In [89]:
# insert data to database
area_table.to_sql('area', engine, if_exists='append', index=False)

19